# 1. Import Ntuple and DecayHash

In [ ]:
import root_pandas
import uproot
import decayHash
from decayHash import DecayHashMap
import sys
import numpy as np
import ROOT
import pandas

# read in root-file as a pandas dataframe
filename='../Ntuples/bengal_generator_50k.root'
decayhash='../Ntuples/hashmap_bengal_generator_50k.root'
#data = uproot.open(filename)['B0'].arrays(library="pd")
data = root_pandas.read_root(filename,key='B0')
MC_e = root_pandas.read_root(filename,key='e')
MC_nu = root_pandas.read_root(filename,key='nu')
hashmap = DecayHashMap(decayhash, removeRadiativeGammaFlag=False)
hashmap2 = DecayHashMap(decayhash, removeRadiativeGammaFlag=True)

In [1]:
import root_pandas
import uproot
import decayHash
from decayHash import DecayHashMap
import sys
import numpy as np
import ROOT
import pandas

# read in root-file as a pandas dataframe
filename='../Ntuples/bengal_e_50k.root'
decayhash='../Ntuples/hashmap_bengal_e_50k.root'
#data = uproot.open(filename)['B0'].arrays(library="pd")
data = root_pandas.read_root(filename,key='B0')
hashmap2 = DecayHashMap(decayhash, removeRadiativeGammaFlag=True)

data['B0_mcPDG'] = data['B0_mcPDG'].fillna(0)

Welcome to JupyROOT 6.24/00


In [9]:
a = [0,-511]
data.query(f'B0_mcPDG in {a}')['B0_mcPDG'].head(10)

0    -511.0
1    -511.0
3    -511.0
4    -511.0
6    -511.0
7    -511.0
13   -511.0
17   -511.0
18   -511.0
20   -511.0
Name: B0_mcPDG, dtype: float64

In [ ]:
import pandas
pandas.set_option('display.max_rows', None)
print(data.isna().sum())

### MC

In [ ]:
# select events where B_tag does not SL decay
MC_e_2 = MC_e[['__event__','__candidate__','E','px','py','pz','M']]
MC_nu_2 = MC_nu[['__event__','__candidate__','E','px','py','pz','M']]

# combine and keep events with inner products of the 4 df
MC_2 = pandas.merge(MC_e_2,MC_nu_2,on=['__event__','__candidate__'],validate='1:1',suffixes=['_e','_nu'])
MC = MC_2.query('__candidate__==0')
MC.eval('q2_MC = M_e**2 + 2*(E_e*E_nu - px_e*px_nu - py_e*py_nu - pz_e*pz_nu)', inplace=True)

# 2. Add DecayMode column to the Ntuple

In [2]:
from collections import OrderedDict

def found(modes,row):
    for mode in modes:
        if mode.startswith(str(int(row['B0_mcPDG']))):
            decaytree = ROOT.Belle2.DecayTree(mode)
            if hashmap2.get_original_decay(row["B0_DecayHash"],row["B0_DecayHashEx"]).find_decay(decaytree):
                return True
        else:
            continue
    return False

def decay_mode(row):
    for name,modes in mode_dict.items():
        if found(modes,row):
            return name
    return 'bkg' # other_B_decay (hadronic) or hadronic_tau or other_D_decay or charged_B or continuum

# the order of keys might be important, try to keep the muon modes at the bottom for e reconstruction
# the e modes will be kept at the bottom for a muon reconstruction
mode_dict = OrderedDict()
mode_dict['sig_D_tau_nu']=['511 (-> -411 (-> 321 -211 -211) -15 (-> -11 12 -16) 16)',
                           '-511 (-> 411 (-> -321 211 211) 15 (-> 11 -12 16) -16)',
                           '511 (-> -411 (-> 321 -211 -211) -15 (-> -13 14 -16) 16)',
                           '-511 (-> 411 (-> -321 211 211) 15 (-> 13 -14 16) -16)']

mode_dict['sig_D_e_nu']=['511 (-> -411 (-> 321 -211 -211) -11 12)',
                         '-511 (-> 411 (-> -321 211 211) 11 -12)']

mode_dict['sig_Dst_tau_nu']=['511 (-> -413 (-> -411 (-> 321 -211 -211) 111) -15 (-> -11 12 -16) 16)',
                             '-511 (-> 413 (-> 411 (-> -321 211 211) 111) 15 (-> 11 -12 16) -16)',
                             '511 (-> -413 (-> -411 (-> 321 -211 -211) 111) -15 (-> -13 14 -16) 16)',
                             '-511 (-> 413 (-> 411 (-> -321 211 211) 111) 15 (-> 13 -14 16) -16)',
                             '511 (-> -413 (-> -411 (-> 321 -211 -211) 22) -15 (-> -11 12 -16) 16)',
                             '-511 (-> 413 (-> 411 (-> -321 211 211) 22) 15 (-> 11 -12 16) -16)',
                             '511 (-> -413 (-> -411 (-> 321 -211 -211) 22) -15 (-> -13 14 -16) 16)',
                             '-511 (-> 413 (-> 411 (-> -321 211 211) 22) 15 (-> 13 -14 16) -16)']

mode_dict['sig_Dst_e_nu']=['511 (-> -413 (-> -411 (-> 321 -211 -211) 111) -11 12)',
                           '511 (-> -413 (-> -411 (-> 321 -211 -211) 22) -11 12)',
                           '-511 (-> 413 (-> 411 (-> -321 211 211) 111) 11 -12)',
                           '-511 (-> 413 (-> 411 (-> -321 211 211) 22) 11 -12)']

mode_dict['all_Dstst_tau_nu']=['511 (-> -10413 -15 (-> -11 12 -16) 16)','-511 (-> 10413 15 (-> 11 -12 16) -16)',
                               '511 (-> -10411 -15 (-> -11 12 -16) 16)','-511 (-> 10411 15 (-> 11 -12 16) -16)',
                               '511 (-> -20413 -15 (-> -11 12 -16) 16)','-511 (-> 20413 15 (-> 11 -12 16) -16)',
                               '511 (-> -415 -15 (-> -11 12 -16) 16)',  '-511 (-> 415 15 (-> 11 -12 16) -16)',
                               '521 (-> -10423 -15 (-> -11 12 -16) 16)','-521 (-> 10423 15 (-> 11 -12 16) -16)',
                               '521 (-> -10421 -15 (-> -11 12 -16) 16)','-521 (-> 10421 15 (-> 11 -12 16) -16)',
                               '521 (-> -20423 -15 (-> -11 12 -16) 16)','-521 (-> 20423 15 (-> 11 -12 16) -16)',
                               '521 (-> -425 -15 (-> -11 12 -16) 16)',  '-521 (-> 425 15 (-> 11 -12 16) -16)',
                               '511 (-> -10413 -15 (-> -13 14 -16) 16)','-511 (-> 10413 15 (-> 13 -14 16) -16)',
                               '511 (-> -10411 -15 (-> -13 14 -16) 16)','-511 (-> 10411 15 (-> 13 -14 16) -16)',
                               '511 (-> -20413 -15 (-> -13 14 -16) 16)','-511 (-> 20413 15 (-> 13 -14 16) -16)',
                               '511 (-> -415 -15 (-> -13 14 -16) 16)',  '-511 (-> 415 15 (-> 13 -14 16) -16)',
                               '521 (-> -10423 -15 (-> -13 14 -16) 16)','-521 (-> 10423 15 (-> 13 -14 16) -16)',
                               '521 (-> -10421 -15 (-> -13 14 -16) 16)','-521 (-> 10421 15 (-> 13 -14 16) -16)',
                               '521 (-> -20423 -15 (-> -13 14 -16) 16)','-521 (-> 20423 15 (-> 13 -14 16) -16)',
                               '521 (-> -425 -15 (-> -13 14 -16) 16)',  '-521 (-> 425 15 (-> 13 -14 16) -16)']

mode_dict['all_Dstst_e_nu']=['511 (-> -10413 -11 12)','-511 (-> 10413 11 -12)',
                             '511 (-> -10411 -11 12)','-511 (-> 10411 11 -12)',
                             '511 (-> -20413 -11 12)','-511 (-> 20413 11 -12)',
                             '511 (-> -415 -11 12)',  '-511 (-> 415 11 -12)',
                             '511 (-> -411 221 -11 12)','-511 (-> 411 221 11 -12)',
                             '511 (-> -411 111 -11 12)','-511 (-> 411 111 11 -12)',
                             '511 (-> -411 111 111 -11 12)','-511 (-> 411 111 111 11 -12)',
                             '511 (-> -411 211 -211 -11 12)','-511 (-> 411 211 -211 11 -12)',
                             '511 (-> -413 221 -11 12)','-511 (-> 413 221 11 -12)',
                             '511 (-> -413 111 -11 12)','-511 (-> 413 111 11 -12)',
                             '511 (-> -413 111 111 -11 12)','-511 (-> 413 111 111 11 -12)',
                             '511 (-> -413 211 -211 -11 12)','-511 (-> 413 211 -211 11 -12)',
                             '511 (-> -421 -211 -11 12)','-511 (-> 421 211 11 -12)',
                             '511 (-> -423 -211 -11 12)','-511 (-> 423 211 11 -12)',
                             '521 (-> -10423 -11 12)','-521 (-> 10423 11 -12)',
                             '521 (-> -10421 -11 12)','-521 (-> 10421 11 -12)',
                             '521 (-> -20423 -11 12)','-521 (-> 20423 11 -12)',
                             '521 (-> -425 -11 12)',  '-521 (-> 425 11 -12)',
                             '521 (-> -411 211 -11 12)','-521 (-> 411 211 11 -12)',
                             '521 (-> -411 211 111 -11 12)','-521 (-> 411 211 111 11 -12)',
                             '521 (-> -413 211 -11 12)','-521 (-> 413 211 11 -12)',
                             '521 (-> -413 211 111 -11 12)','-521 (-> 413 211 111 11 -12)']

mode_dict['sig_D_mu_nu']=['511 (-> -411 (-> 321 -211 -211) -13 14)',
                          '-511 (-> 411 (-> -321 211 211) 13 -14)']

mode_dict['sig_Dst_mu_nu']=['511 (-> -413 (-> -411 (-> 321 -211 -211) 111) -13 14)',
                           '511 (-> -413 (-> -411 (-> 321 -211 -211) 22) -13 14)',
                           '-511 (-> 413 (-> 411 (-> -321 211 211) 111) 13 -14)',
                           '-511 (-> 413 (-> 411 (-> -321 211 211) 22) 13 -14)']

mode_dict['all_Dstst_mu_nu']=['511 (-> -10413 -13 14)','-511 (-> 10413 13 -14)',
                              '511 (-> -10411 -13 14)','-511 (-> 10411 13 -14)',
                              '511 (-> -20413 -13 14)','-511 (-> 20413 13 -14)',
                              '511 (-> -415 -13 14)',  '-511 (-> 415 13 -14)',
                              '511 (-> -411 221 -13 14)','-511 (-> 411 221 13 -14)',
                              '511 (-> -411 111 -13 14)','-511 (-> 411 111 13 -14)',
                              '511 (-> -411 111 111 -13 14)','-511 (-> 411 111 111 13 -14)',
                              '511 (-> -411 211 -211 -13 14)','-511 (-> 411 211 -211 13 -14)',
                              '511 (-> -413 221 -13 14)','-511 (-> 413 221 13 -14)',
                              '511 (-> -413 111 -13 14)','-511 (-> 413 111 13 -14)',
                              '511 (-> -413 111 111 -13 14)','-511 (-> 413 111 111 13 -14)',
                              '511 (-> -413 211 -211 -13 14)','-511 (-> 413 211 -211 13 -14)',
                              '511 (-> -421 -211 -13 14)','-511 (-> 421 211 13 -14)',
                              '511 (-> -423 -211 -13 14)','-511 (-> 423 211 13 -14)',
                              '521 (-> -10423 -13 14)','-521 (-> 10423 13 -14)',
                              '521 (-> -10421 -13 14)','-521 (-> 10421 13 -14)',
                              '521 (-> -20423 -13 14)','-521 (-> 20423 13 -14)',
                              '521 (-> -425 -13 14)',  '-521 (-> 425 13 -14)',
                              '521 (-> -411 211 -13 14)','-521 (-> 411 211 13 -14)',
                              '521 (-> -411 211 111 -13 14)','-521 (-> 411 211 111 13 -14',
                              '521 (-> -413 211 -13 14)','-521 (-> 413 211 13 -14)',
                              '521 (-> -413 211 111 -13 14)','-521 (-> 413 211 111 13 -14)']

In [14]:
cut = 'D_vtxReChi2<3 and B0_vtxReChi2<14 and -3.2<B0_deltaE<0 and e_CMS_p>0.2 and \
    5<B0_roeMbc_my_mask and 4.3<B0_CMS2_weMbc and \
    -5<B0_roeDeltae_my_mask<2 and -3<B0_CMS0_weDeltae<2 and \
    abs(B0_roeCharge_my_mask)<3 and B0_CMS_E<5.4 and \
    0.2967<B0_Lab5_weMissPTheta<2.7925 and 0.2967<B0_Lab6_weMissPTheta<2.7925 and \
    0<B0_TagVReChi2<100 and 0<B0_TagVReChi2IP<100'
data = data.query(cut).copy()
data['isSignal'] = 1.0
data.eval('B_D_ReChi2 = B0_vtxReChi2 + D_vtxReChi2', inplace=True)
data.eval('p_D_l = D_CMS_p + e_CMS_p', inplace=True)

# Calculate the q2 in different ways
data.eval('q2_SigOnly = (0.5*Ecms - D_CMS_E)**2 - D_CMS_px**2 - D_CMS_py**2 - D_CMS_pz**2', inplace=True)
data.eval('MM2_SigOnly = (0.5*Ecms - D_CMS_E - e_CMS_E)**2 - (D_CMS_px + e_CMS_px)**2 - (D_CMS_py + e_CMS_py)**2 - (D_CMS_pz + e_CMS_pz)**2', inplace=True)

data['DecayMode'] = data.apply(decay_mode, axis=1).astype('category') #axis=0 will allow the application to be done at a column
data.DecayMode.value_counts()

sig_D_e_nu    10191
bkg             701
Name: DecayMode, dtype: int64

In [15]:
cut = 'D_vtxReChi2<3 and B0_vtxReChi2<14 and -3.2<B0_deltaE<0 and e_CMS_p>0.2 and \
    5<B0_roeMbc_my_mask and 4.3<B0_CMS2_weMbc and \
    -5<B0_roeDeltae_my_mask<2 and -3<B0_CMS0_weDeltae<2 and \
    abs(B0_roeCharge_my_mask)<3 and B0_CMS_E<5.4 and \
    0.2967<B0_Lab5_weMissPTheta<2.7925 and 0.2967<B0_Lab6_weMissPTheta<2.7925 and \
    0<B0_TagVReChi2<100 and 0<B0_TagVReChi2IP<100'
#cut='B0_TauminusMode==-1'#'DecayMode=="bkg" and B0_isContinuumEvent!=1'
candidate12 = data.query(cut).iloc[112][['B0_DecayHash', "B0_DecayHashEx"]].values

# print the original decay as simulated in MC with removed Bremsstrahlung gammas
print("Monte Carlo Decay with removed Bremsstrahlung gammas: ")
org2 = hashmap2.get_original_decay(*candidate12)
print(org2.to_string())

Monte Carlo Decay with removed Bremsstrahlung gammas: 
300553
    511
        -411
            313
                311
                    310
                111
                    22
                    22
            -211
            111
                22
                22
        211
        113
            211
            -211
        311
            130
        -311
            130
    -511
        411
            -321
            211
            211
        11
        -12



In [ ]:
samples = {}
names = ['BC','AC']
cut = 'D_vtxReChi2<13 and B0_vtxReChi2<14 and -3.2<B0_deltaE<0 and e_CMS_p>0.2 and \
    5<B0_roeMbc_my_mask and 4.3<B0_CMS2_weMbc and \
    -5<B0_roeDeltae_my_mask<2 and -3<B0_CMS0_weDeltae<2 and \
    abs(B0_roeCharge_my_mask)<3 and nElectrons90+nMuons90==1'
for name in names:
    if name == 'BC':
        df = data.copy()
    else:
        df = data.query(cut).copy()
    
    print(f'{name} before BCS')
    print(df.DecayMode.value_counts())

    df_bestSelected=df.loc[df.groupby(['__experiment__','__run__','__event__','__production__']).B_D_ReChi2.idxmin()]

    print(f'{name} after BCS')
    print(df_bestSelected.DecayMode.value_counts())
    
    df_merged = pandas.merge(df_bestSelected,MC,on=['__event__'],validate='1:1')
    samples[name] = df_merged

# 4. Get fitting templates

In [ ]:
# apply BDTs
# plot mm2, mm2 vs p_D_l
# save the templates

In [ ]:
import basf2_mva
import pandas

identifier_1 = '/home/belle/zhangboy/R_D/Generic_MC14rd/Continuum_Suppression/MVA1_FastBDT.xml'
test_1 = '../Ntuples/bengal_eidglobal_50k_cut.root'
data.query(cut).to_root(test_1, key='B0')
output_file_1 = '../Ntuples/bengal_eidglobal_50k_MVA1.root'

identifier_1_5 = '/home/belle/zhangboy/R_D/Generic_MC14rd/B_bkg_Suppression/MVA1_5/MVA1_5_FastBDT.xml'
test_1_5 = '../Ntuples/bengal_eidglobal_50k_cut.root'
output_file_1_5 = '../Ntuples/bengal_eidglobal_50k_MVA1_5.root'

output_file_1_5_applied = '../Ntuples/bengal_eidglobal_50k_MVA1_5_applied.root'

identifier_2_1 = '/home/belle/zhangboy/R_D/Generic_MC14rd/B_bkg_Suppression/MVA2/MVA2_1_FastBDT.xml'
test_2_1 = output_file_1_5_applied
output_file_2_1 = '../Ntuples/bengal_eidglobal_50k_MVA2_1.root'
output_file_2_1_applied = '../Ntuples/bengal_eidglobal_50k_MVA2_1_applied.root'

In [ ]:
# apply CS BDT identifier_1, merge data file and mva output, rename the column
basf2_mva.expert(basf2_mva.vector(identifier_1),  # weightfile
                 basf2_mva.vector(test_1),
                 'B0', output_file_1)

df1 = data.query(cut).drop_duplicates(subset=['__experiment__','__run__','__event__','__production__','__candidate__']).reset_index(drop=True)
df2 = root_pandas.read_root(output_file_1)
print(len(df1)==len(df2))
df_1 = pandas.concat([df1,df2],axis=1)

df_1=df_1.rename(columns={"__slhome__slbelle__slzhangboy__slR_D__slGeneric_MC14rd__slContinuum_Suppression__slMVA1_FastBDT__ptxml": "MVA1_output"})
df_1=df_1.drop(columns=['__slhome__slbelle__slzhangboy__slR_D__slGeneric_MC14rd__slContinuum_Suppression__slMVA1_FastBDT__ptxml_isSignal'])

# apply BDT 1_5 identifier_1_5, merge, rename, change the output type, save
basf2_mva.expert(basf2_mva.vector(identifier_1_5),  # weightfile
                 basf2_mva.vector(test_1_5),
                 'B0', output_file_1_5)

df3 = root_pandas.read_root(output_file_1_5)
print(len(df_1)==len(df3))
df_1_5 = pandas.concat([df_1,df3],axis=1)

df_1_5=df_1_5.rename(columns={"__slhome__slbelle__slzhangboy__slR_D__slGeneric_MC14rd__slB_bkg_Suppression__slMVA1_5__slMVA1_5_FastBDT__ptxml": "MVA1_5_output"})
df_1_5=df_1_5.drop(columns=['__slhome__slbelle__slzhangboy__slR_D__slGeneric_MC14rd__slB_bkg_Suppression__slMVA1_5__slMVA1_5_FastBDT__ptxml_isSignal'])

df_1_5.MVA1_5_output=np.float64(df_1_5.MVA1_5_output)
print(type(df_1_5.MVA1_5_output[0]))
print(type(df_1_5.isSignal[0]))

df_1_5.to_root(output_file_1_5_applied, key='B0')

# apply BDT 2_1 identifier_2_1, merge, rename, save
basf2_mva.expert(basf2_mva.vector(identifier_2_1),  # weightfile
                 basf2_mva.vector(test_2_1),
                 'B0', output_file_2_1)

df4 = root_pandas.read_root(output_file_2_1)
print(len(df_1_5)==len(df4))
df_2_1 = pandas.concat([df_1_5, df4],axis=1)
print(len(df_1_5)==len(df_2_1))

df_2_1=df_2_1.rename(columns={"__slhome__slbelle__slzhangboy__slR_D__slGeneric_MC14rd__slB_bkg_Suppression__slMVA2__slMVA2_1_FastBDT__ptxml": "MVA2_1_output"})
df_2_1=df_2_1.drop(columns=['__slhome__slbelle__slzhangboy__slR_D__slGeneric_MC14rd__slB_bkg_Suppression__slMVA2__slMVA2_1_FastBDT__ptxml_isSignal'])

df_2_1.to_root(output_file_2_1_applied, key='B0')

In [ ]:
df_2_1.columns
len(df_2_1)

In [ ]:
df_bestSelected=df_2_1.loc[df_2_1.groupby(['__experiment__','__run__','__event__','__production__']).B_D_ReChi2.idxmin()]

In [ ]:
import pandas
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
plt.style.use('belle2')
# plot mm2, mm2 vs p_D_l
# save the templates

In [ ]:
# Signal components
sig_D_e_nu=samples['AC'].query('DecayMode=="sig_D_e_nu" and B0_mcErrors<16 and abs(D_mcPDG)==411 and abs(e_genMotherPDG)==511').copy()
sig_D_tau_nu=samples['AC'].query('DecayMode=="sig_D_tau_nu" and B0_mcErrors<32 and abs(D_mcPDG)==411 and abs(e_genMotherPDG)==15').copy()
sig_Dst_e_nu=samples['AC'].query('DecayMode=="sig_Dst_e_nu" and B0_mcErrors<64 and abs(D_mcPDG)==411 and abs(e_genMotherPDG)==511').copy()
sig_Dst_tau_nu=samples['AC'].query('DecayMode=="sig_Dst_tau_nu" and B0_mcErrors<64 and abs(D_mcPDG)==411 and abs(e_genMotherPDG)==15').copy()
all_Dstst_e_nu=samples['AC'].query('DecayMode=="all_Dstst_e_nu" and abs(D_mcPDG)==411 and \
((B0_mcErrors<64 and abs(e_genMotherPDG)==511) or (B0_mcErrors<512 and abs(e_genMotherPDG)==521))').copy()
all_Dstst_tau_nu=samples['AC'].query('DecayMode=="all_Dstst_tau_nu" and abs(D_mcPDG)==411 and abs(e_genMotherPDG)==15 and \
((B0_mcErrors<64 and abs(B0_mcPDG)==511) or (B0_mcErrors<512 and abs(B0_mcPDG)==521))').copy()

#sig_D_mu_nu=samples['AC'].query('DecayMode=="sig_D_mu_nu" and B0_mcErrors<16').copy()
#sig_Dst_mu_nu=samples['AC'].query('DecayMode=="sig_Dst_mu_nu" and (16<=B0_mcErrors<32 or B0_mcErrors<8)').copy()
#all_Dstst_mu_nu=samples['AC'].query('DecayMode=="all_Dstst_mu_nu" and (16<=B0_mcErrors<64 or B0_mcErrors<8)').copy()

# Bkg components
bkg_fakeD = samples['AC'].query('abs(D_mcPDG)!=411 and B0_mcErrors!=512 and B0_isContinuumEvent!=1').copy()
bkg_fakeTracksClusters = samples['AC'].query('B0_mcErrors==512 and B0_isContinuumEvent!=1').copy()
bkg_fakeDTC = pandas.concat([bkg_fakeD, bkg_fakeTracksClusters])

bkg_combinatorial = samples['AC'].query('B0_mcPDG==300553 and abs(D_mcPDG)==411 and B0_mcErrors!=512 and B0_isContinuumEvent!=1').copy()
bkg_sigOtherBDTaudecay = samples['AC'].query('(DecayMode=="bkg" or DecayMode=="sig_D_mu_nu" or DecayMode=="sig_Dst_mu_nu" or DecayMode=="all_Dstst_mu_nu") and \
B0_mcPDG!=300553 and abs(D_mcPDG)==411 and B0_mcErrors!=512 and B0_isContinuumEvent!=1').copy()
bkg_fakeB = pandas.concat([bkg_combinatorial, bkg_sigOtherBDTaudecay])

bkg_continuum = samples['AC'].query('B0_isContinuumEvent==1').copy()

bkg_others = pandas.concat([samples['AC'],
                           sig_D_e_nu,
                           sig_D_tau_nu,
                           sig_Dst_e_nu,
                           sig_Dst_tau_nu,
                           all_Dstst_e_nu,
                           all_Dstst_tau_nu,
                           bkg_fakeDTC,
                           bkg_fakeB,
                           bkg_continuum]).drop_duplicates(keep=False)
# Weird! the bkg_others contains some events with
# correct sig decay hash chain and correct B0_mcPDG, D_mcPDG, e_genMotherPDG,
# but with 128< B0_mcErrors < 256 (misID)


def statistics(df):
    counts=df.count()
    mean=df.mean()
    std=df.std()
    return f'''counts = %d \nmean = %5.3f \nstd = %5.3f''' %(counts,mean,std)

kwarg={'bins':50, 'histtype':'step','lw':2}

    
def plot_projection(cut,variable):
    (counts1, bins1) = np.histogram(sig_D_tau_nu.query(cut)[variable], bins=50)
    (counts2, bins2) = np.histogram(sig_Dst_tau_nu.query(cut)[variable], bins=50)
    factor = 1
    plt.hist(bins1[:-1], bins1, weights=factor*counts1,label=f'D_tau_nu \n{statistics(sig_D_tau_nu.query(cut)[variable])}',alpha=0.6)
    sig_D_e_nu.query(cut)[variable].hist(label=f'D_e_nu \n{statistics(sig_D_e_nu.query(cut)[variable])}',**kwarg)
    
    plt.hist(bins2[:-1], bins2, weights=factor*counts2,label=f'Dst_tau_nu \n{statistics(sig_Dst_tau_nu.query(cut)[variable])}',alpha=0.6,histtype='step',lw=2)
    sig_Dst_e_nu.query(cut)[variable].hist(label=f'Dst_e_nu \n{statistics(sig_Dst_e_nu.query(cut)[variable])}',**kwarg)
    all_Dstst_tau_nu.query(cut)[variable].hist(label=f'all_Dstst_tau_nu \n{statistics(all_Dstst_tau_nu.query(cut)[variable])}',**kwarg)
    all_Dstst_e_nu.query(cut)[variable].hist(label=f'all_Dstst_e_nu \n{statistics(all_Dstst_e_nu.query(cut)[variable])}',**kwarg)
    bkg_fakeDTC.query(cut)[variable].hist(label=f'bkg_fakeD_Tracks_Clusters \n{statistics(bkg_fakeDTC.query(cut)[variable])}',**kwarg)
    bkg_fakeB.query(cut)[variable].hist(label=f'bkg_comb_wrongDecay \n{statistics(bkg_fakeB.query(cut)[variable])}',**kwarg)
    bkg_continuum.query(cut)[variable].hist(label=f'bkg_continuum \n{statistics(bkg_continuum.query(cut)[variable])}',**kwarg)
    bkg_others.query(cut)[variable].hist(label=f'bkg_others \n{statistics(bkg_others.query(cut)[variable])}',**kwarg)
    plt.legend(bbox_to_anchor=(1,1),ncol=3, fancybox=True, shadow=True,labelspacing=1.5)

def plot_q2_efficiency(cut, q2='B0_CMS3_weQ2lnuSimple'):
    sig_D_e_nu_BC=samples['BC'].query('DecayMode=="sig_D_e_nu" and B0_mcErrors<16 and abs(D_mcPDG)==411 and abs(e_genMotherPDG)==511').copy()
    sig_D_tau_nu_BC=samples['BC'].query('DecayMode=="sig_D_tau_nu" and B0_mcErrors<32 and abs(D_mcPDG)==411 and abs(e_genMotherPDG)==15').copy()
    (counts1, bins1) = np.histogram(sig_D_e_nu_BC.query(cut)[q2], bins=15)
    (counts2, bins2) = np.histogram(sig_D_e_nu.query(cut)[q2], bins=bins1)
    
    efficiency = counts2 / counts1
    efficiency_err = efficiency * np.sqrt(1/counts1 + 1/counts2)
    #factor = 1
    #plt.hist(bins1[:-1], bins1, weights=factor*efficiency,label='D_l_nu efficiency in q2',histtype='step')
    bin_centers = (bins1[:-1] + bins1[1:]) /2
    plt.errorbar(x=bin_centers, y=efficiency, yerr=efficiency_err, fmt='ko',label='D_l_nu efficiency in q2')
    plt.legend()
    
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.tab20.colors)

In [ ]:
plot_projection('MVA1_output>0.4 and MVA2_1_output>0.2 and B0_roeMbc_my_mask>5.26', 'B0_CMS2_weMissM2')
plt.xlabel("CMS2_weMissM2 $[GeV^2/c^4]$")
plt.ylabel('# of counts per bin')
plt.title('CMS2_weMissM2');
#plt.xlim(-10,10)

In [ ]:
sig_sample = all_Dstst_tau_nu

cut='B0_roeMbc_my_mask>5.26 and MVA1_output>0.4 and MVA2_1_output>0.2'
xedges = np.linspace(0.5, 4, 15)
yedges = np.linspace(-4, 10, 25)
variable_x = 'p_D_l'
variable_y = 'B0_CMS2_weMissM2'

(counts, xedges, yedges) = np.histogram2d(sig_sample.query(cut)[variable_x], 
                                          sig_sample.query(cut)[variable_y],
                                          bins=[xedges, yedges])
counts = counts.T + 0.01
fig,axs=plt.subplots(ncols=1,figsize=(10,4))
X, Y = np.meshgrid(xedges, yedges)
im=axs.pcolormesh(X, Y, counts, cmap='jet', norm=colors.LogNorm())
axs.grid()
fig.colorbar(im)

In [ ]:
import json

workspace_file = '/home/belle/zhangboy/R_D/Signal_MC_ROEx1/2d_2channels_workspace.json'
with open(workspace_file, 'r+') as f:
    data = json.load(f)
    data['channels'][0]['samples'][4]['name'] = 'Dstst_tau_nu'
    data['channels'][0]['samples'][4]['data'] = counts.ravel().tolist()
    # counts.ravel()/.reshape(-1) returns a view, counts.flatten() returns a copy (slower)
    f.seek(0)        # <--- should reset file position to the beginning.
    json.dump(data, f, indent=4)
    f.truncate()     # remove remaining part

In [ ]:
plot_q2_efficiency(cut='-2<B0_CMS3_weQ2lnuSimple<13')
plt.xlabel("q2 $[GeV^2/c^4]$")
plt.ylabel('Cuts Efficiency')
plt.title('Dl$\\nu$ cut efficiency vs. q2');
plt.xlim(-2,12);
plt.ylim(0,1);

In [ ]:
plot_projection('B0_roeMbc_my_mask>5', 'B0_CMS2_weMissM2')
plt.xlabel("CMS2_weMissM2 $[GeV^2/c^4]$")
plt.ylabel('# of counts per bin')
plt.title('MissM2 with E*_roe=$\sqrt{s}/2$')
plt.xlim(-8,8);

In [ ]:
plot_projection('B0_roeMbc_my_mask>5', 'B0_CMS3_weMissM2')
plt.xlabel("CMS3_weMissM2 $[GeV^2/c^4]$")
plt.ylabel('# of counts per bin')
plt.title('MissM2 with E*_roe=$\sqrt{s}/2$ and p*_roe=0')
plt.xlim(-8,8);

In [ ]:
plot_projection('B0_roeMbc_my_mask>5', 'B0_CMS4_weMissM2')
plt.xlabel("CMS4_weMissM2 $[GeV^2/c^4]$")
plt.ylabel('# of counts per bin')
plt.title('MissM2 with E*_roe=$\sqrt{s}/2$ and p*_roe=0.34GeV + direction');
plt.xlim(-8,8);

In [ ]:
plot_projection('B0_roeMbc_my_mask>5', 'q2_MC')
plt.xlabel("q2 $[GeV^2/c^4]$")
plt.ylabel('# of counts per bin')
plt.title('MC True q2')
plt.xlim(-2,13);

In [ ]:
plot_projection('B0_roeMbc_my_mask>5', 'B0_CMS2_weQ2lnuSimple')
plt.xlabel("CMS2_q2 $[GeV^2/c^4]$")
plt.ylabel('# of counts per bin')
plt.title('q2 with E*_roe=$\sqrt{s}/2$')
plt.xlim(-8,13);

In [ ]:
plot_projection('B0_roeMbc_my_mask>5', 'B0_CMS3_weQ2lnuSimple')
plt.xlabel("CMS2_q2 $[GeV^2/c^4]$")
plt.ylabel('# of counts per bin')
plt.title('q2 with E*_roe=$\sqrt{s}/2$ and p*_roe=0')
plt.xlim(-2,13);

In [ ]:
plot_projection('B0_roeMbc_my_mask>5', 'B0_CMS4_weQ2lnuSimple')
plt.xlabel("CMS4_q2 $[GeV^2/c^4]$")
plt.ylabel('# of counts per bin')
plt.title('q2 with E*_roe=$\sqrt{s}/2$ and p*_roe=0.34GeV + direction');
plt.xlim(-4,13);

In [ ]:
plt.scatter(sig_D_e_nu.q2_MC, sig_D_e_nu.B0_CMS2_weQ2lnuSimple,label='q2')
x = np.linspace(-0.5, 12, 10)
plt.plot(x,x,color='red',label='y=x')
plt.xlabel("MC True q2 $[GeV^2/c^4]$")
plt.ylabel('Reconstructed q2')
plt.title('q2 with E*_roe=$\sqrt{s}/2$')
plt.legend();

In [ ]:
plt.scatter(sig_D_e_nu.q2_MC, sig_D_e_nu.B0_CMS3_weQ2lnuSimple,label='q2')
x = np.linspace(-0.5, 12, 10)
plt.plot(x,x,color='red',label='y=x')
plt.xlabel("MC True q2 $[GeV^2/c^4]$")
plt.ylabel('Reconstructed q2')
plt.title('q2 with E*_roe=$\sqrt{s}/2$ and p*_roe=0')
plt.legend();

In [ ]:
plt.scatter(sig_D_e_nu.q2_MC, sig_D_e_nu.B0_CMS4_weQ2lnuSimple,label='q2')
x = np.linspace(-0.5, 12, 10)
plt.plot(x,x,color='red',label='y=x')
plt.xlabel("MC True q2 $[GeV^2/c^4]$")
plt.ylabel('Reconstructed q2')
plt.title('q2 with E*_roe=$\sqrt{s}/2$ and p*_roe=0.34GeV + direction')
plt.legend();

In [ ]:
plot_projection('q2_MC<2', 'B0_CMS3_weQ2lnuSimple')
plt.xlabel("CMS2_q2 $[GeV^2/c^4]$")
plt.ylabel('# of counts per bin')
plt.title('q2 with E*_roe=$\sqrt{s}/2$ and p*_roe=0')
plt.xlim(-2,13);

In [ ]:
plot_projection('2<q2_MC<4', 'B0_CMS3_weQ2lnuSimple')
plt.xlabel("CMS2_q2 $[GeV^2/c^4]$")
plt.ylabel('# of counts per bin')
plt.title('q2 with E*_roe=$\sqrt{s}/2$ and p*_roe=0')
plt.xlim(-2,13);

In [ ]:
plot_projection('4<q2_MC<6', 'B0_CMS3_weQ2lnuSimple')
plt.xlabel("CMS2_q2 $[GeV^2/c^4]$")
plt.ylabel('# of counts per bin')
plt.title('q2 with E*_roe=$\sqrt{s}/2$ and p*_roe=0')
plt.xlim(-2,13);

In [ ]:
plot_projection('6<q2_MC<8', 'B0_CMS3_weQ2lnuSimple')
plt.xlabel("CMS2_q2 $[GeV^2/c^4]$")
plt.ylabel('# of counts per bin')
plt.title('q2 with E*_roe=$\sqrt{s}/2$ and p*_roe=0')
plt.xlim(-2,13);

In [ ]:
plot_projection('8<q2_MC<10', 'B0_CMS3_weQ2lnuSimple')
plt.xlabel("CMS2_q2 $[GeV^2/c^4]$")
plt.ylabel('# of counts per bin')
plt.title('q2 with E*_roe=$\sqrt{s}/2$ and p*_roe=0')
plt.xlim(-2,13);

In [ ]:
plot_projection('10<q2_MC', 'B0_CMS3_weQ2lnuSimple')
plt.xlabel("CMS2_q2 $[GeV^2/c^4]$")
plt.ylabel('# of counts per bin')
plt.title('q2 with E*_roe=$\sqrt{s}/2$ and p*_roe=0')
plt.xlim(-2,13);